## アプリ概要

この Streamlit アプリは、ARC-AGI 形式のパズルタスクを手作業で解くためのテスト環境です。  
サイドバーから問題データ（JSON）を選択し、入出力の例をビジュアルに確認しながら、自分で実装した Python 関数 `d(g)` をその場で実行・検証できます。

- **タスク選択**：`task001.json` ～ `task400.json` をプルダウンから選択  
- **例の可視化**：train/test の入出力をカラーマップ付きグリッドで表示  
- **コードエディタ**：streamlit-ace を使った埋め込みエディタで `d(g)` を自由に編集  
- **即時テスト**：▶️ ボタンで全例を実行し、OK/NG を表示。最後に全例正解ならacceptメッセージ
```
.
├── README.md
├── app.py
└── google-code-golf-2025/
    ├── task001.json
    ├── task002.json
    ├── …  
    └── task400.json
google-code-golf-2025/ 配下に各タスクの JSON ファイル（taskXXX.json）を置いてください。
```
---

## App Overview

This Streamlit app provides an interactive testing environment for manually solving ARC-AGI–style puzzle tasks.
You can select a problem dataset (JSON) from the sidebar, inspect its input/output examples visually, and instantly execute and validate your own Python function d(g).

Task Selector: Choose task001.json through task400.json from a dropdown

Example Visualization: Render train/test inputs and outputs as color-mapped grids

Code Editor: Edit the solver function d(g) inline using streamlit-ace

Instant Testing: Click the ▶️ button to run all examples and see ✅/❌ for each; if all pass, an “accepted” message is shown

```.
├── README.md
├── app.py
└── google-code-golf-2025/
    ├── task001.json
    ├── task002.json
    ├── …  
    └── task400.json
```
Place all puzzle JSON files (taskXXX.json) under the google-code-golf-2025/ directory.

In [ ]:
!pip install streamlit streamlit-ace matplotlib

In [ ]:
# # -*- coding: utf-8 -*-
# import os
# import json
# import streamlit as st
# from glob import glob
# from streamlit_ace import st_ace
# import io, contextlib
# import matplotlib.pyplot as plt

# # --- 多言語対応ラベル ---
# LABELS = {
#     "en": {
#         "title": "ARC-AGI Code Tester",
#         "select_task": "Select a task",
#         "loaded": "Loaded",
#         "examples": "Examples",
#         "train_input": "Train Input",
#         "train_output": "Train Output",
#         "test_input": "Test Input",
#         "your_solution": "Your Solution",
#         "run_test": "▶️ Run & Test",
#         "input": "Input",
#         "your_output": "Your Output",
#         "expected": "Expected",
#         "ok": "✅ OK",
#         "ng": "❌ NG",
#         "all_passed": "🎉 All examples passed! 🎉",
#         "error_func": "⚠️ Function `d` is not defined.",
#         "error_exec": "⚠️ Error during execution:"
#     },
#     "ja": {
#         "title": "ARC-AGI コード テスター",
#         "select_task": "タスクを選択",
#         "loaded": "をロード",
#         "examples": "例",
#         "train_input": "Train 入力",
#         "train_output": "Train 出力",
#         "test_input": "Test 入力",
#         "your_solution": "あなたの解答",
#         "run_test": "▶️ 実行＆検証",
#         "input": "入力",
#         "your_output": "あなたの出力",
#         "expected": "期待される出力",
#         "ok": "✅ OK",
#         "ng": "❌ NG",
#         "all_passed": "🎉 全ての例をパスしました！ 🎉",
#         "error_func": "⚠️ 関数 `d` が定義されていません。",
#         "error_exec": "⚠️ 実行中にエラーが発生しました:"
#     }
# }

# # --- 言語選択 ---
# lang = st.sidebar.selectbox("Language / 言語", options=["ja", "en"], index=0)
# l = LABELS[lang]

# st.set_page_config(layout="wide")
# st.title(l["title"])

# # --- サイドバー: タスク選択 ---
# st.sidebar.markdown(f"**{l['select_task']}**")

# task_paths = sorted(glob("google-code-golf-2025/task*.json"))
# task_names = [os.path.basename(p) for p in task_paths]
# selected_name = st.sidebar.selectbox("タスクを選択", task_names)
# # map the chosen filename back to its full path
# selected = task_paths[ task_names.index(selected_name) ]
# with open(selected) as f:
#     task = json.load(f)
# st.sidebar.markdown(f"**{selected_name} {l['loaded']}**")

# # --- グリッド描画関数 ---
# def show_grid(grid, caption=None):
#     cmap = plt.get_cmap("tab10")
#     fig, ax = plt.subplots(figsize=(3,3))
#     ax.imshow(grid, cmap=cmap, vmin=0, vmax=9)
#     ax.set_xticks([])
#     ax.set_yticks([])
#     if caption:
#         ax.set_title(caption, fontsize=10)
#     st.pyplot(fig)
#     plt.close(fig)

# # --- 例の表示 ---
# st.subheader(l["examples"])
# train = task.get("train", [])
# test  = task.get("test", [])
# n = max(len(train), len(test))
# cols = st.columns(n)
# for i, col in enumerate(cols):
#     with col:
#         st.markdown(f"### Example {i+1}")
#         if i < len(train):
#             st.markdown(f"**{l['train_input']}**")
#             show_grid(train[i]["input"])
#             st.markdown(f"**{l['train_output']}**")
#             show_grid(train[i]["output"])
#         if i < len(test):
#             st.markdown(f"**{l['test_input']}**")
#             show_grid(test[i]["input"])

# # --- コード入力エリア ---
# st.subheader(l["your_solution"])
# def_code = """\
# # def p(g): を実装してください
# def p(g):
#     return g
# """
# user_code = st_ace(
#     value=def_code,
#     language="python",
#     theme="monokai",
#     key="ace",
#     font_size=14,
#     height=250,
# )

# # --- 実行＆検証 ---
# if st.button(l['run_test']):
#     buf = io.StringIO()
#     ns = {}
#     try:
#         with contextlib.redirect_stdout(buf):
#             exec(user_code, ns)
#         fn = ns.get("p")
#         if not callable(fn):
#             st.error(l['error_func'])
#         else:
#             results = {"train": [], "test": [], "arc-gen": []}
#             for split in ["train","test","arc-gen"]:
#                 for ex in task.get(split, []):
#                     out = fn(ex["input"])
#                     ok = (out == ex["output"])
#                     results[split].append({
#                         "input": ex["input"],
#                         "expected": ex["output"],
#                         "output": out,
#                         "ok": ok
#                     })
#             all_ok = all(item["ok"] for split in ["train","test"] for item in results[split])
#             stdout = buf.getvalue()
#             if stdout.strip():
#                 st.subheader("Stdout")
#                 st.code(stdout)
#             for split in ["train","test"]:
#                 if not results[split]: continue
#                 st.subheader(split.upper())
#                 cols = st.columns(len(results[split]))
#                 for col, res in zip(cols, results[split]):
#                     with col:
#                         st.markdown(f"**{l['input']}**")
#                         show_grid(res["input"])
#                         st.markdown(f"**{l['your_output']}**")
#                         show_grid(res["output"])
#                         st.markdown(f"**{l['expected']}**")
#                         show_grid(res["expected"])
#                         if res["ok"]:
#                             st.success(l['ok'])
#                         else:
#                             st.error(l['ng'])
#             if all_ok:
#                 st.success(l['all_passed'])
#     except Exception as e:
#         st.error(f"{l['error_exec']} {e}")
